In [74]:
from burstextractor.burstlist import download_burst_list, process_burst_list
from burstextractor.timeutils import extract_time, fix_typos_in_time, fix_24_hour_time, create_datetime, check_valid_date, adjust_year_month
from burstextractor.data_utils import explode_instruments_long_clean_instruments, keep_only_type_I_to_VI
from spectogram_utils import spec_to_pd_dataframe, plot_spectogram
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt 
from radiospectra.sources import CallistoSpectrogram

In [75]:
burst_list = pd.read_excel('burst_list.xlsx')

In [76]:
burst_list.instruments.value_counts()

Australia-ASSA         1876
Arecibo-Observatory    1307
HUMAIN                 1245
GLASGOW                1185
ALASKA-COHOE           1168
SWISS-Landschlacht     1162
ALASKA-HAARP           1045
BIR                     956
INDIA-UDAIPUR           903
INDIA-OOTY              901
AUSTRIA-UNIGRAZ         844
SWISS-HEITERSWIL        743
INDIA-GAURI             731
SWISS-MUHEN             552
MRO                     489
EGYPT-Alexandria        488
ALMATY                  421
TRIEST                  394
GREENLAND               391
SOUTHAFRICA-SANSA       356
SPAIN-PERALEJOS         346
SWISS-IRSOL             342
MEXICO-LANCE            320
ROSWELL-NM              286
DENMARK                 258
INDONESIA               255
MRT1                    251
USA-ARIZONA-ERAU        244
MEXART                  221
AUSTRIA-OE3FLB          215
MEXICO-LANCE-B          202
SSRT                    196
MEXICO-LANCE-A          194
MONGOLIA-UB             184
KRIM                    162
HURBANOVO           

In [77]:
instruments = ['Australia-ASSA']

In [78]:
burst_list_filtered = burst_list[burst_list.instruments.isin(instruments)]

In [79]:
min_time_flare = burst_list_filtered.datetime_start.dt.time.min()

In [80]:
max_time_flare = burst_list_filtered.datetime_end.dt.time.max()

In [81]:
burst_list_filtered.datetime_end.median()

Timestamp('2022-04-27 03:27:00')

In [82]:
MIN_TIME_TO_FLARE = datetime.timedelta(minutes=30)

In [83]:
def generate_random_date(min_date, max_date):
    """
    This function will generate a random datetime between two datetime
    objects.
    """
    delta = max_date - min_date
    int_delta = (delta.days * 24 * 60 * 60) + delta.seconds
    random_second = np.random.randint(int_delta)
    return min_date + datetime.timedelta(seconds=random_second)

In [84]:
burst_list_filtered

,date,time,type,instruments,time_start,time_end,date_start,date_end,datetime_start,datetime_end
0,20210119,02:42-02:42,III,Australia-ASSA,02:42,02:42,20210119,20210119,2021-01-19 02:42:00,2021-01-19 02:42:00
7,20210127,04:32-04:32,III,Australia-ASSA,04:32,04:32,20210127,20210127,2021-01-27 04:32:00,2021-01-27 04:32:00
89,20210419,06:55-06:57,III,Australia-ASSA,06:55,06:57,20210419,20210419,2021-04-19 06:55:00,2021-04-19 06:57:00
108,20210419,23:39-23:42,II,Australia-ASSA,23:39,23:42,20210419,20210419,2021-04-19 23:39:00,2021-04-19 23:42:00
109,20210419,23:39-23:43,III,Australia-ASSA,23:39,23:43,20210419,20210419,2021-04-19 23:39:00,2021-04-19 23:43:00
...,...,...,...,...,...,...,...,...,...,...
21961,20230111,23:08-23:09,III,Australia-ASSA,23:08,23:09,20230111,20230111,2023-01-11 23:08:00,2023-01-11 23:09:00
21962,20230111,23:43-23:43,III,Australia-ASSA,23:43,23:43,20230111,20230111,2023-01-11 23:43:00,2023-01-11 23:43:00
21963,20230112,00:04-00:04,III,Australia-ASSA,00:04,00:04,20230112,20230112,2023-01-12 00:04:00,2023-01-12 00:04:00
21964,20230112,00:55-01:00,III,Australia-ASSA,00:55,01:00,20230112,20230112,2023-01-12 00:55:00,2023-01-12 01:00:00


In [85]:
non_burst_list = []
while len(non_burst_list) <= 1876:
    for instrument in instruments:
        df = pd.DataFrame()
        burst_list_filtered = burst_list[burst_list.instruments == instrument]
        df['datetime_start'] = [generate_random_date(burst_list_filtered.datetime_start.min(), burst_list_filtered.datetime_end.max())]
        df['date'] = [df.datetime_start.dt.date[0]]
        df['time'] = [df.datetime_start.dt.time[0]]
        df['instruments'] = [instrument]
        df['type'] = ['non_burst']
        non_burst_list.append(df)
    
non_burst_list = pd.concat(non_burst_list)
non_burst_list.reset_index(inplace=True, drop=True)

In [86]:
non_burst_list

,datetime_start,date,time,instruments,type
0,2022-02-23 08:52:26,2022-02-23,08:52:26,Australia-ASSA,non_burst
1,2021-11-26 22:33:56,2021-11-26,22:33:56,Australia-ASSA,non_burst
2,2021-05-21 14:04:38,2021-05-21,14:04:38,Australia-ASSA,non_burst
3,2021-05-29 06:38:22,2021-05-29,06:38:22,Australia-ASSA,non_burst
4,2022-10-18 06:28:21,2022-10-18,06:28:21,Australia-ASSA,non_burst
...,...,...,...,...,...
1872,2021-06-29 00:46:24,2021-06-29,00:46:24,Australia-ASSA,non_burst
1873,2022-07-27 21:07:22,2022-07-27,21:07:22,Australia-ASSA,non_burst
1874,2022-11-22 07:09:33,2022-11-22,07:09:33,Australia-ASSA,non_burst
1875,2022-05-06 20:13:59,2022-05-06,20:13:59,Australia-ASSA,non_burst


In [87]:
non_burst_list.to_excel('non_burst_list.xlsx')